In [1]:
import pandas as pd
import os
import yaml

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from data_loader import *
from utils import *

In [2]:
# Config
config_path = 'config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)


In [3]:
# Mysql string
db_url = config['MYSQL_STRING']

In [4]:
def write_df_to_db(df, table_name, db_type="postgres", db_url=None, if_exists="append", chunksize=5000):
    """
    Write a pandas DataFrame to either PostgreSQL or MySQL based on db_type.
    
    Args:
        df (pd.DataFrame): DataFrame to write
        table_name (str): Table name
        db_type (str): "postgres" or "mysql"
        db_url (str): Full SQLAlchemy DB URL
        if_exists (str): 'append', 'replace', or 'fail'
        chunksize (int): Number of rows per batch
    """
    import pandas as pd
    from sqlalchemy import create_engine
    from sqlalchemy.orm import sessionmaker

    if db_url is None:
        raise ValueError("db_url must be provided")

    # Convert unsupported types
    for col in df.columns:
        if pd.api.types.is_period_dtype(df[col]):
            df[col] = df[col].dt.to_timestamp()
        elif pd.api.types.is_timedelta64_dtype(df[col]):
            df[col] = df[col].dt.total_seconds()
        elif pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype(str)

    # Ensure the driver is correct
    if db_type.lower() == "postgres":
        # PostgreSQL usually uses psycopg2
        if "postgresql" not in db_url:
            db_url = "postgresql+psycopg2://" + db_url.split("://")[1]
    elif db_type.lower() == "mysql":
        # MySQL usually uses pymysql
        if "mysql" not in db_url:
            db_url = "mysql+pymysql://" + db_url.split("://")[1]
    else:
        raise ValueError("db_type must be 'postgres' or 'mysql'")

    # Create engine and session
    engine = create_engine(db_url)
    Session = sessionmaker(bind=engine)

    # Write using session manager
    with Session() as session:
        try:
            df.to_sql(
                table_name,
                con=session.connection(),
                if_exists=if_exists,
                index=False,
                chunksize=chunksize
            )
            session.commit()
        except Exception as e:
            session.rollback()
            print(f"Error writing to table '{table_name}': {e}")
            raise

In [5]:
# League dat
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']].sort_values('league_id')

,league_id,league_name,country_name
13,1,World Cup,World
17,2,UEFA Champions League,World
179,3,UEFA Europa League,World
0,4,Euro Championship,World
359,5,UEFA Nations League,World
...,...,...,...
1126,1129,ASEAN Club Championship,World
1127,1130,USL Super League,USA
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World


In [6]:
#write_df_to_db(leagues_dat.iloc[:,1:], "league", db_type="mysql", db_url=db_url)

In [7]:
teams_dat = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")
teams_dat.head()

,team_name,league
0,Angers,61
1,Lille,61
2,Lyon,61
3,Marseille,61
4,Nantes,61


In [8]:
# Add Team-legaue map to db
#write_df_to_db(teams_dat, "team_league_map", db_type="mysql", db_url=db_url)

In [9]:
fixtures_dir = home_dir + "/data/Fixtures"

complete_data = pd.DataFrame()
for file in os.listdir(fixtures_dir):
    dat = pd.read_parquet(os.path.join(fixtures_dir,file))
    complete_data = pd.concat([complete_data,dat],axis = 0)

complete_data = complete_data.reset_index()
complete_data.drop(columns = ['index'],inplace=True)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_5149/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_5149/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_5149/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenatio

In [10]:
# Data checks
complete_data['passes_accuracy'] = complete_data['passes_accuracy'].astype("float64")
complete_data.rename(columns= {'passes_accuracy':'passes_accurate'},inplace =True)
complete_data['fixture_date'] = pd.to_datetime(complete_data.fixture_date)
complete_data['fixture_date_dt'] = complete_data['fixture_date'].dt.date
complete_data = create_datetime_columns(complete_data,'fixture_date')
complete_data['games_rating'] = pd.to_numeric(complete_data['games_rating'])

complete_data['season'] = complete_data['fixture_date'].apply(get_season)


# Targets
complete_data['outcome_num'] = pd.Categorical(complete_data.outcome).codes

complete_data['win'] = np.where(complete_data.outcome.str.lower() == 'win', 1,0)
complete_data['draw'] = np.where(complete_data.outcome.str.lower() == 'draw', 1,0)
complete_data['loss'] = np.where(complete_data.outcome.str.lower() == 'loss', 1,0)

# # primary position map:
# player_map = get_major_position(complete_data)

# # Join back to complete_data
# complete_data = pd.merge(complete_data,player_map,on = 'player_id',how = 'left')

# Joins:
complete_data = complete_data.merge(teams_dat.drop_duplicates(),how = 'left', left_on= 'team',right_on = 'team_name').drop(columns = ['team_name'])
complete_data = complete_data.merge(leagues_dat[['league_id','league_name']],how = 'left', left_on = 'league', right_on = 'league_id')


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:50: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['month_e'] = data[dt_col].dt.to_period('M')
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:54: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['week_e'] = data[dt_col].dt.to_period('W-MON')


In [24]:
stored_fixtures = pd.read_sql("select distinct fixture_id,team from overperformxg.complete_data",config['MYSQL_STRING'])
stored_fixtures['stored'] = 'True'
all_fixtures = complete_data[['fixture_id','team']].drop_duplicates()
all_fixtures['stored'] = 'True'
new_fixtures = all_fixtures.merge(stored_fixtures,on = ('fixture_id','team'),how = 'left',suffixes=('_new','_db'))
new_fixtures = new_fixtures[(new_fixtures.stored_db.isna()) & (new_fixtures.stored_new == 'True')]
new_fixtures

,fixture_id,team,stored_new,stored_db
21837,1378975,Wolves,True,NaN
21838,1378998,Wolves,True,NaN
21839,1379018,Wolves,True,NaN
21840,1379038,Wolves,True,NaN
21841,1379058,Wolves,True,NaN
21842,1379088,Wolves,True,NaN
21843,1379108,Wolves,True,NaN
21844,1379118,Wolves,True,NaN
21845,1378980,Wolves,True,NaN
21846,1379007,Wolves,True,NaN


In [25]:
complete_data = complete_data[complete_data.fixture_id.isin(new_fixtures.fixture_id)]

In [26]:
complete_data.shape

(320, 66)

In [27]:
write_df_to_db(complete_data,"complete_data",db_type="mysql", db_url=db_url)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_5149/2079050629.py:22: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df[col]):
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_5149/2079050629.py:26: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[col]):
